In [ ]:
!pip install datasets
!pip install --upgrade sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from datasets import Dataset, load_dataset
import pandas as pd
import ast
import re


In [ ]:
from datasets import Dataset

def expand_dataset(df):
    expanded_data = {
        "anchor": [],
        "positive": []
    }

    # Duyệt qua từng dòng trong dataframe gốc
    for index, row in train.iterrows():
        expanded_data['anchor'].append(row['question'])
        expanded_data['positive'].append(row['context'])

    # Chuyển dữ liệu mở rộng thành Dataset
    return Dataset.from_dict(expanded_data)

# Sử dụng
train = pd.read_csv("/kaggle/input/preprocessed-train-24dec/preprocessed_train_24Dec.csv")
train_dataset = expand_dataset(train)
subset = train_dataset.select(range(80000)) 

In [ ]:
print(len(subset))

In [ ]:
# subset = train_dataset.select(range(40)) 
# for item in subset:
#     print(item['anchor'])
#     print(item['positive'])
#     break

In [ ]:
# !pip install flash-attn --no-build-isolation
# !pip install triton

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer(
#     "BAAI/bge-m3"
# )

# model = model.to('cuda')
# print(model.device)
# loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("hiieu/halong_embedding")

model = model.to('cuda')
print(model.device)
loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
model

In [ ]:
loss

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
print("entering training cell ")

In [ ]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# from sentence_transformers.training_args import BatchSamplers
# LR = 2e-05
# BS = 2
# GRAD_ACC_STEP = 1
# REPORT_TO = "none"
# args = SentenceTransformerTrainingArguments(
#     # Required parameter:
#     output_dir="/kaggle/working/bge_finetune_24Dec",
#     num_train_epochs=1,
#     per_device_train_batch_size=BS,
#     gradient_accumulation_steps=GRAD_ACC_STEP,
#     per_device_eval_batch_size=BS,
#     eval_accumulation_steps=GRAD_ACC_STEP,
#     learning_rate=LR,
#     warmup_ratio=0.1,
#     fp16=True,
#     bf16=False, 
#     batch_sampler=BatchSamplers.NO_DUPLICATES,  # Losses using "in-batch negatives" benefit from no duplicates
#     # Optional tracking/debugging parameters:
#     save_strategy="steps",
#     save_steps=100,
#     save_total_limit=2,
#     logging_steps=100,
#     auto_find_batch_size=False,
#     do_eval = False,
#     report_to=REPORT_TO
# )
# trainer = SentenceTransformerTrainer(
#     model=model,
#     args=args,
#     train_dataset=subset,
#     loss=loss,
# )


In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="/kaggle/working/bi_finetune_26Dec", # output directory and hugging face model ID
    num_train_epochs=1,                         # number of epochs
    per_device_train_batch_size=8,             # train batch size
    gradient_accumulation_steps=4,             # for a global batch size of 512
    per_device_eval_batch_size=4,              # evaluation batch size
    #gradient_checkpointing=True,
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    #tf32=True,                                  # use tf32 precision
    bf16=True,                                  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    save_strategy="steps",
    # eval_strategy="steps",                      # evaluate after each epoch
    save_steps = 100,
    logging_steps=100,                           # log every 10 steps
    save_total_limit=2,                         # save only the last 3 models
    metric_for_best_model="eval_dim_768_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
    report_to="none"
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args, 
    train_dataset=subset,
    loss=loss,
    # evaluator=evaluator,
)

In [ ]:
print("start training")

In [ ]:
trainer.train()


In [ ]:
# model_new = SentenceTransformer(
#     "/kaggle/working/bge_finetune_25Dec/checkpoint-1250"
# )

In [ ]:
# query = "Người học ngành quản lý khai thác công trình thủy lợi trình độ cao đẳng phải có khả năng học tập và nâng cao trình độ như thế nào?"
# query_embedding = model_new.encode(query)

# corpus = "Khả năng học tập, nâng cao trình độ Khối lượng kiến thức tối thiểu, yêu cầu về năng lực mà người học phải đạt được sau khi tốt nghiệp ngành, nghề quản lý, khai thác các công trình thủy lợi, trình độ cao đẳng có thể tiếp tục phát triển ở các trình độ cao hơn. Người học sau tốt nghiệp có năng lực tự học, tự cập nhật những tiến bộ khoa học công nghệ trong phạm vi ngành, nghề để nâng cao trình độ hoặc học liên thông lên trình độ cao hơn trong cùng ngành, nghề hoặc trong nhóm ngành, nghề hoặc trong cùng lĩnh vực đào tạo"
# corpus_embedding = model_new.encode(corpus)

# corpus_2 = "Giới thiệu chung về ngành, nghề Quản lý, khai thác các công trình thủy lợi trình độ cao đẳng là ngành, nghề phục vụ tưới, tiêu, dân sinh, công nghiệp, nông nghiệp, an ninh quốc phòng, đáp ứng yêu cầu bậc 5 trong Khung trình độ quốc gia Việt Nam. Quản lý khai thác các công trình thủy lợi trình độ cao đẳng bao gồm các nhiệm vụ chính như: Quan trắc khí tượng thủy văn; trắc đạc công trình thủy lợi; quan trắc công trình thủy lợi; quản lý vận hành, khai thác tưới, cấp, tiêu và thoát nước; quản lý vận hành, khai thác công trình thủy lợi đầu mối; quản lý vận hành, khai thác kênh và công trình trên kênh; thi công tu bổ công trình thủy lợi; duy tu bảo dưỡng công trình thủy lợi; phòng chống lụt bão; lập, lưu trữ hồ sơ quản lý công trình; bảo vệ công trình thủy lợi; giám sát an toàn lao động và vệ sinh môi trường, Người hành nghề quản lý, khai thác công trình thủy lợi thường làm việc tại các doanh nghiệp quản lý, khai thác công trình thủy lợi, doanh nghiệp khai thác tài nguyên nước... họ cần có đủ kiến thức, kỹ năng, sức khỏe để làm việc ở văn phòng, công trình hoặc ngoài trời, đôi khi phải làm việc trong những điều kiện khắc nghiệt như gió bão, lũ lụt… Khối lượng kiến thức tối thiểu: 2.500 giờ (tương đương 90 tín chỉ)."
# corpus_embedding_2 = model_new.encode(corpus_2)
# score = model_new.similarity(query_embedding, corpus_embedding)

# print(score[0].item())
# print(" ")
# score = model_new.similarity(query_embedding, corpus_embedding_2)

# print(score[0].item())

In [ ]:
# import shutil

# # Replace with your folder path
# folder_path = '/kaggle/working/bge_finetune_25Dec'
# output_zip = '/kaggle/working/25Dec.zip'

# # Create the ZIP file
# shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_path)

# # Make the ZIP file available for download
# from IPython.display import FileLink
# FileLink(output_zip)
